In [23]:
categorias = {
    "BEBIDAS": ["BEBIDA CALIENTE", "BEBIDA FRIA"],
    "BINES": ["BINES"],
    "COMIDA": ["COMIDA CONGELADA", "COMIDA REFRIGERADA"],
    "CUMPLEAÑOS": ["B-DAY"],
    "JUEGOS": ["JUEGOS"],
    "OTROS": ["ASEO", "PAPEL"],
    "PREMIOS": ["PREMIOS"],
}

In [37]:
import csv
import io

import pandas as pd


def procesar_reporte_inventario(contenido_archivo):
    """
    Procesa el contenido de un reporte de inventario en formato de texto
    y lo convierte en un DataFrame de Pandas.

    El script maneja encabezados de página, pies de página, categorías,
    subcategorías y líneas de datos que usan comillas para los números
    con comas decimales.

    (Versión corregida para alinear columnas)
    """

    # 1. Definición de Columnas
    nombres_columnas_datos = [
        "Nombre Materia Prima",
        "IdArt",
        "UnidInv",
        "InvInic",
        "UnidComp",
        "InvFinal",
        "CostxUnid",
        "TotalFinal",
        "Días a la Mano",
        "UsoUnid_Real",
        "UsoUnid_Teórico",
        "UsoUnid_Variación",
        "ValorUso_Real",
        "ValorUso_Teórico",
        "ValorUso_Variación",
        "PorcUso_Real",
        "PorcUso_Teórico",
        "PorcUso_Variación",
    ]

    columnas_finales = ["Categoria", "Subcategoria"] + nombres_columnas_datos

    # 2. Inicialización de variables
    datos_procesados = []
    categoria_actual = None
    subcategoria_actual = None

    lineas = contenido_archivo.splitlines()

    # 3. Iteración línea por línea
    for linea in lineas:
        linea = linea.strip()

        # --- REGLAS DE OMISIÓN (JUNK) ---
        if (
            not linea
            or linea.startswith("Copyright")
            or linea.startswith("*El Cálculo")
            or linea.startswith("Theoretical")
            or linea.startswith("V 21.1.158.0")
            or "3243.CEC .LaFlorida" in linea
            or linea.startswith("UsoenUnid,Valor $ Uso")
            or linea.startswith("Nombre Materia Prima")
            or linea.startswith("IdArt,UnidInv")
            or linea.startswith("VentasNetas:")
        ):
            continue

        # --- REGLAS DE ESTADO (CATEGORÍAS) ---
        if linea.startswith("Total:"):
            if linea.startswith("Total:,") and "," in linea:
                partes = linea.split(",")
                if len(partes) > 1:
                    nombre_total = partes[1].strip()
                    if nombre_total == subcategoria_actual:
                        subcategoria_actual = None
                    elif nombre_total == categoria_actual:
                        categoria_actual = None
                        subcategoria_actual = None
            continue

        if (
            linea.isupper()
            and "," not in linea
            and "$" not in linea
            and not any(c.isdigit() for c in linea)
        ):
            if categoria_actual is None:
                categoria_actual = linea
            elif subcategoria_actual is None or subcategoria_actual == categoria_actual:
                subcategoria_actual = linea
            else:
                if categoria_actual == subcategoria_actual:
                    categoria_actual = linea
                    subcategoria_actual = None
                else:
                    subcategoria_actual = linea

            continue

        # --- PROCESAMIENTO DE DATOS ---
        try:
            f = io.StringIO(linea)
            reader = csv.reader(f)
            campos = next(reader)

            # Asegurarnos de que sea una línea de datos válida (tiene 20 campos)
            if len(campos) == 20:
                # ==================================================
                # --- AQUÍ ESTÁ LA LÍNEA CORREGIDA ---
                #
                # Tomamos campos 0-3
                # Saltamos campo 4 (columna vacía de UnidComp)
                # Tomamos campos 5-6 (datos de UnidComp e InvFinal)
                # Saltamos campo 7 (columna vacía)
                # Tomamos campos 8-19 (el resto de los datos)

                fila_datos = campos[0:4] + campos[5:7] + campos[8:20]
                # ==================================================

                # Añadimos la categoría y subcategoría
                fila_completa = [categoria_actual, subcategoria_actual] + fila_datos

                datos_procesados.append(fila_completa)

        except Exception:
            # Ignora líneas que no se puedan procesar
            continue

    # 4. Creación del DataFrame
    print(
        f"Procesamiento finalizado. Se encontraron {len(datos_procesados)} registros."
    )

    if not datos_procesados:
        print("No se encontraron datos para crear el DataFrame.")
        return pd.DataFrame(columns=columnas_finales)

    df = pd.DataFrame(datos_procesados, columns=columnas_finales)

    return df

In [39]:
# 1. Define el nombre de tu archivo .txt
#    (Asegúrate de que coincida exactamente con el nombre de tu archivo)
nombre_del_archivo_txt = "2.02 Resumen costo inventario. (FIFO).txt"

print(f"Iniciando la lectura del archivo: {nombre_del_archivo_txt}")

# 2. Bloque para leer el archivo de forma segura
try:
    # Usamos 'with open(...)' porque maneja el archivo de forma segura
    # 'r' significa modo "lectura" (read)
    #
    # CAMBIO: Usamos 'utf-16-le' (o 'utf-16') en lugar de 'utf-8'
    # Esto es necesario por el formato en que el sistema guardó el .txt
    with open(nombre_del_archivo_txt, "r", encoding="utf-16-le") as f:
        # f.read() lee el archivo completo y lo guarda en una variable
        contenido_completo = f.read()

    print("Archivo leído con éxito. Procesando contenido...")

    # 3. ¡Aquí ejecutamos la función!
    #    Le pasamos el contenido que acabamos de leer.
    df_resultado = procesar_reporte_inventario(contenido_completo)

    # 4. Mostramos los resultados para verificar
    print("\n--- ¡DataFrame creado exitosamente! ---")
    print("\n--- Primeros 5 registros: ---")
    print(df_resultado.head(5))

    print("\n--- Información del DataFrame (columnas y tipos de datos): ---")
    df_resultado.info()

    # Opcional: Si quieres guardar este DataFrame limpio en un
    # archivo CSV para abrirlo en Excel, descomenta la siguiente línea:

    df_resultado.to_csv("inventario_procesado.csv", index=False)
    print("\n(Opcional: El DataFrame se ha guardado como 'inventario_procesado.csv')")

except FileNotFoundError:
    print(
        f"¡ERROR! No se pudo encontrar el archivo: {'2.02 Resumen costo inventario. (FIFO).txt'}"
    )
    print("Por favor, revisa que el nombre esté escrito exactamente igual")
    print("y que el archivo .txt esté en la misma carpeta que este script.")
except Exception as e:
    print(f"Ocurrió un error inesperado durante la lectura: {e}")

Iniciando la lectura del archivo: 2.02 Resumen costo inventario. (FIFO).txt
Archivo leído con éxito. Procesando contenido...
Procesamiento finalizado. Se encontraron 364 registros.

--- ¡DataFrame creado exitosamente! ---

--- Primeros 5 registros: ---
  Categoria     Subcategoria Nombre Materia Prima    IdArt UnidInv   InvInic  \
0   BEBIDAS  BEBIDA CALIENTE           Cafe Grano  1000083      GR   7.549,0   
1   BEBIDAS  BEBIDA CALIENTE           Chai Latte  1000084      GR   6.785,0   
2   BEBIDAS  BEBIDA CALIENTE            Chocolate  1000085      GR  10.680,0   
3   BEBIDAS  BEBIDA CALIENTE     Leche Descremada  1000086      GR   7.630,0   
4   BEBIDAS  BEBIDA CALIENTE           Revolvedor  1000087      UN   1.594,0   

  UnidComp InvFinal CostxUnid   TotalFinal Días a la Mano UsoUnid_Real  \
0  7.256,0  9.063,0    $41,67  $377.638,32           48,9     5.742,00   
1      0,0  4.720,0    $17,28   $81.561,60           70,9     2.065,00   
2      0,0  7.360,0    $11,66   $85.847,04  